In [ ]:
from urllib import request, parse
from http.cookiejar import CookieJar
import numpy
import getpass
import netrc
import requests
import json
import boto3
import os
import boto3
import matplotlib.pyplot as plt
import io
from urllib.parse import urlparse
import sys  
!{sys.executable} -m pip install --user jsonlines
import jsonlines
from PIL import Image

session = boto3.Session(profile_name='sandbox')
client = session.client('s3')

bucket = 'dnewman2-pwm6dtmajjx83833e31rt99qeg1d4usw2a-s3alias'
sourceRef='s3://' + bucket


# Creating an augemented manifest file (AMF)
We use json lines format to produce our input data for training and validation.

Each line references an image and a class describing whether it is cloudy (1) or clear (0)

For example,

`{"source-ref": "s3://lp-prod-public/HLSL30.020/HLS.L30.T01FBF.2021104T213801.v2.0/HLS.L30.T01FBF.2021104T213801.v2.0.jpg", "class": "1"}`

We construct this file by rendering the image and asking the user whether the image is cloudy or not. Then we generate an entry in the AMF and move on to the next image from our list objects array

In [ ]:
def createManifestFile(f, bucket, response):
    with jsonlines.Writer(f) as writer:
        client
        for key in response['Contents']:
            s3_loc = key['Key']
            if s3_loc.endswith('.jpg'):
                response = client.get_object(
                    Bucket=bucket,
                    Key=s3_loc)
                # Render image
                file_stream = response['Body']
                img = Image.open(file_stream)
                imgplot = plt.imshow(img)
                plt.show(imgplot)       
                # Ask for classification           
                while True:
                    try:
                        cloudy = int(input("Cloudy? [1:cloudy, 0:not cloudy]:"))
                        if cloudy < 0 or cloudy > 1:
                            raise ValueError          
                        data = {}
                        data['source-ref'] = 's3://' + bucket + '/' + s3_loc
                        data['class'] = str(cloudy)
                        writer.write(data)        
                        break
                    except ValueError:
                        print("Invalid input. The number must be in the range of 0-1.")  
    f.close()

# Generate training augmented manifest file
Get the first 80 objects in the bucket and generate our training dataset

In [ ]:



f = open("cloud_training.json",'w')
response = client.list_objects_v2(
    Bucket=bucket,
    MaxKeys=100,
)

createManifestFile(f, bucket, response)
token = response['NextContinuationToken']

# Write the file to our S3 bucket

In [ ]:

with open("cloud_training.json", "rb") as f:
    client.put_object(
        Bucket='dug-cloud-manifest',
        Key='cloud_training.json',
        Body=f
    )
    f.close()

# Generate validation augmented manifest file
Get the next 10 objects in the bucket and generate our validation dataset

In [ ]:
# Generate validation augmented manifest file
response = client.list_objects_v2(
    Bucket=bucket,
    MaxKeys=100,
    ContinuationToken=token
)
f = open("cloud_validation.json",'w')
createManifestFile(f, bucket, response)

Write the file to our S3 bucket

In [ ]:
with open("cloud_validation.json", "rb") as f:
    client.put_object(
        Bucket='dug-cloud-manifest',
        Key='cloud_validation.json',
        Body=f
    )
    f.close()

In [ ]:
print('Done!')